In [594]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from dateutil.parser import parse
from countryinfo import CountryInfo

# Data Tidying and Cleaning Lab
## Reading, tidying and cleaning data. Preparing data for exploration, mining, analysis and learning

In this lab, you'll be working with the Coffee Quality Index dataset, located [here](https://www.kaggle.com/datasets/volpatto/coffee-quality-database-from-cqi). For convenience (and to save trouble in case you can't download files, or someone uploads a newer version), I've provided the dataset in the `data/` folder. The metadata (description) is at the Kaggle link. For this lab, you'll only need `merged_data_cleaned.csv`, as it is the concatenation of the other two datasets.

In this (and the following labs), you'll get several questions and problems. Do your analysis, describe it, use any tools and plots you wish, and answer. You can create any amount of cells you'd like.

Sometimes, the answers will not be unique, and they will depend on how you decide to approach and solve the problem. This is usual - we're doing science after all!

It's a good idea to save your clean dataset after all the work you've done to it.

### Problem 1. Read the dataset (1 point)
This should be self-explanatory. The first column is the index.

In [595]:
data = pd.read_csv('data/merged_data_cleaned.csv')

In [596]:
data

,Unnamed: 0,Species,Owner,Country.of.Origin,Farm.Name,Lot.Number,Mill,ICO.Number,Company,Altitude,...,Color,Category.Two.Defects,Expiration,Certification.Body,Certification.Address,Certification.Contact,unit_of_measurement,altitude_low_meters,altitude_high_meters,altitude_mean_meters
0,0,Arabica,metad plc,Ethiopia,metad plc,NaN,metad plc,2014/2015,metad agricultural developmet plc,1950-2200,...,Green,0,"April 3rd, 2016",METAD Agricultural Development plc,309fcf77415a3661ae83e027f7e5f05dad786e44,19fef5a731de2db57d16da10287413f5f99bc2dd,m,1950.0,2200.0,2075.0
1,1,Arabica,metad plc,Ethiopia,metad plc,NaN,metad plc,2014/2015,metad agricultural developmet plc,1950-2200,...,Green,1,"April 3rd, 2016",METAD Agricultural Development plc,309fcf77415a3661ae83e027f7e5f05dad786e44,19fef5a731de2db57d16da10287413f5f99bc2dd,m,1950.0,2200.0,2075.0
2,2,Arabica,grounds for health admin,Guatemala,"san marcos barrancas ""san cristobal cuch",NaN,NaN,NaN,NaN,1600 - 1800 m,...,NaN,0,"May 31st, 2011",Specialty Coffee Association,36d0d00a3724338ba7937c52a378d085f2172daa,0878a7d4b9d35ddbf0fe2ce69a2062cceb45a660,m,1600.0,1800.0,1700.0
3,3,Arabica,yidnekachew dabessa,Ethiopia,yidnekachew dabessa coffee plantation,NaN,wolensu,NaN,yidnekachew debessa coffee plantation,1800-2200,...,Green,2,"March 25th, 2016",METAD Agricultural Development plc,309fcf77415a3661ae83e027f7e5f05dad786e44,19fef5a731de2db57d16da10287413f5f99bc2dd,m,1800.0,2200.0,2000.0
4,4,Arabica,metad plc,Ethiopia,metad plc,NaN,metad plc,2014/2015,metad agricultural developmet plc,1950-2200,...,Green,2,"April 3rd, 2016",METAD Agricultural Development plc,309fcf77415a3661ae83e027f7e5f05dad786e44,19fef5a731de2db57d16da10287413f5f99bc2dd,m,1950.0,2200.0,2075.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1334,1334,Robusta,luis robles,Ecuador,robustasa,Lavado 1,our own lab,NaN,robustasa,NaN,...,Blue-Green,1,"January 18th, 2017",Specialty Coffee Association,ff7c18ad303d4b603ac3f8cff7e611ffc735e720,352d0cf7f3e9be14dad7df644ad65efc27605ae2,m,NaN,NaN,NaN
1335,1335,Robusta,luis robles,Ecuador,robustasa,Lavado 3,own laboratory,NaN,robustasa,40,...,Blue-Green,0,"January 18th, 2017",Specialty Coffee Association,ff7c18ad303d4b603ac3f8cff7e611ffc735e720,352d0cf7f3e9be14dad7df644ad65efc27605ae2,m,40.0,40.0,40.0
1336,1336,Robusta,james moore,United States,fazenda cazengo,NaN,cafe cazengo,NaN,global opportunity fund,795 meters,...,NaN,6,"December 23rd, 2015",Specialty Coffee Association,ff7c18ad303d4b603ac3f8cff7e611ffc735e720,352d0cf7f3e9be14dad7df644ad65efc27605ae2,m,795.0,795.0,795.0
1337,1337,Robusta,cafe politico,India,NaN,NaN,NaN,14-1118-2014-0087,cafe politico,NaN,...,Green,1,"August 25th, 2015",Specialty Coffee Association,ff7c18ad303d4b603ac3f8cff7e611ffc735e720,352d0cf7f3e9be14dad7df644ad65efc27605ae2,m,NaN,NaN,NaN


### Problem 2. Observations and features (1 point)
How many observations are there? How many features? Which features are numerical, and which are categorical?

**Note:** Think about the _meaning_, not the data types. The dataset hasn't been thoroughly cleaned.

### How many observations are there?

In [597]:
print(f"The dataset has {len(data)} observations")

The dataset has 1339 observations


### How many features are in the dataset?

In [598]:
print(f"All features are: {data.columns.value_counts().sum()}")

All features are: 44


### Which features are numerical and which are categorical?

In [599]:
data_types = data.dtypes.reset_index()
data_types = data_types.rename(columns={'index': 'feature', 0: 'date_type'})
data_types

,feature,date_type
0,Unnamed: 0,int64
1,Species,object
2,Owner,object
3,Country.of.Origin,object
4,Farm.Name,object
5,Lot.Number,object
6,Mill,object
7,ICO.Number,object
8,Company,object
9,Altitude,object


##### Numerical features - data that can be manipulated with arithmetic operations
##### Categorical features - data that can't be manipulated with arithmetic operations and present categories, groups, labels

### Problem 3. Column manipulation (1 point)
Make the column names more Pythonic (which helps with the quality and... aesthetics). Convert column names to `snake_case`, i.e. `species`, `country_of_origin`, `ico_number`, etc. Try to not do it manually.

In [600]:
#Replace all "." with "_" in the column name
data.columns = data.columns.str.replace('.', '_')

In [601]:
#Make lower case all column names
data.columns = data.columns.str.lower()

In [602]:
#Change firs column name
data = data.rename(columns={'unnamed: 0': 'id'})

In [603]:
data

,id,species,owner,country_of_origin,farm_name,lot_number,mill,ico_number,company,altitude,...,color,category_two_defects,expiration,certification_body,certification_address,certification_contact,unit_of_measurement,altitude_low_meters,altitude_high_meters,altitude_mean_meters
0,0,Arabica,metad plc,Ethiopia,metad plc,NaN,metad plc,2014/2015,metad agricultural developmet plc,1950-2200,...,Green,0,"April 3rd, 2016",METAD Agricultural Development plc,309fcf77415a3661ae83e027f7e5f05dad786e44,19fef5a731de2db57d16da10287413f5f99bc2dd,m,1950.0,2200.0,2075.0
1,1,Arabica,metad plc,Ethiopia,metad plc,NaN,metad plc,2014/2015,metad agricultural developmet plc,1950-2200,...,Green,1,"April 3rd, 2016",METAD Agricultural Development plc,309fcf77415a3661ae83e027f7e5f05dad786e44,19fef5a731de2db57d16da10287413f5f99bc2dd,m,1950.0,2200.0,2075.0
2,2,Arabica,grounds for health admin,Guatemala,"san marcos barrancas ""san cristobal cuch",NaN,NaN,NaN,NaN,1600 - 1800 m,...,NaN,0,"May 31st, 2011",Specialty Coffee Association,36d0d00a3724338ba7937c52a378d085f2172daa,0878a7d4b9d35ddbf0fe2ce69a2062cceb45a660,m,1600.0,1800.0,1700.0
3,3,Arabica,yidnekachew dabessa,Ethiopia,yidnekachew dabessa coffee plantation,NaN,wolensu,NaN,yidnekachew debessa coffee plantation,1800-2200,...,Green,2,"March 25th, 2016",METAD Agricultural Development plc,309fcf77415a3661ae83e027f7e5f05dad786e44,19fef5a731de2db57d16da10287413f5f99bc2dd,m,1800.0,2200.0,2000.0
4,4,Arabica,metad plc,Ethiopia,metad plc,NaN,metad plc,2014/2015,metad agricultural developmet plc,1950-2200,...,Green,2,"April 3rd, 2016",METAD Agricultural Development plc,309fcf77415a3661ae83e027f7e5f05dad786e44,19fef5a731de2db57d16da10287413f5f99bc2dd,m,1950.0,2200.0,2075.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1334,1334,Robusta,luis robles,Ecuador,robustasa,Lavado 1,our own lab,NaN,robustasa,NaN,...,Blue-Green,1,"January 18th, 2017",Specialty Coffee Association,ff7c18ad303d4b603ac3f8cff7e611ffc735e720,352d0cf7f3e9be14dad7df644ad65efc27605ae2,m,NaN,NaN,NaN
1335,1335,Robusta,luis robles,Ecuador,robustasa,Lavado 3,own laboratory,NaN,robustasa,40,...,Blue-Green,0,"January 18th, 2017",Specialty Coffee Association,ff7c18ad303d4b603ac3f8cff7e611ffc735e720,352d0cf7f3e9be14dad7df644ad65efc27605ae2,m,40.0,40.0,40.0
1336,1336,Robusta,james moore,United States,fazenda cazengo,NaN,cafe cazengo,NaN,global opportunity fund,795 meters,...,NaN,6,"December 23rd, 2015",Specialty Coffee Association,ff7c18ad303d4b603ac3f8cff7e611ffc735e720,352d0cf7f3e9be14dad7df644ad65efc27605ae2,m,795.0,795.0,795.0
1337,1337,Robusta,cafe politico,India,NaN,NaN,NaN,14-1118-2014-0087,cafe politico,NaN,...,Green,1,"August 25th, 2015",Specialty Coffee Association,ff7c18ad303d4b603ac3f8cff7e611ffc735e720,352d0cf7f3e9be14dad7df644ad65efc27605ae2,m,NaN,NaN,NaN


### Problem 4. Bag weight (1 point)
What's up with the bag weights? Make all necessary changes to the column values. Don't forget to document your methods and assumptions.

In [604]:
# Read and check bag_weight. We can remove "kg" in the data
data.bag_weight

0       60 kg
1       60 kg
2           1
3       60 kg
4       60 kg
        ...  
1334     2 kg
1335     2 kg
1336     1 kg
1337    5 lbs
1338    5 lbs
Name: bag_weight, Length: 1339, dtype: object

In [605]:
data.bag_weight_kg.value_counts()

AttributeError: 'DataFrame' object has no attribute 'bag_weight_kg'

In [606]:
#Remove "kg" from data in bag_weight column
# data.bag_weight = data.bag_weight.str.replace('kg', '').str.strip()
# data.bag_weight = data.bag_weight.str.replace('lbs', '').str.strip()
# data.bag_weight = data.bag_weight.str.replace(',', '').str.strip()

#Change column name from "bag_weight" to "bag_weight_kg"
data = data.rename(columns={'bag_weight': 'bag_weight_kg'})

#Read the data to check all changes
data.bag_weight_kg

0       60 kg
1       60 kg
2           1
3       60 kg
4       60 kg
        ...  
1334     2 kg
1335     2 kg
1336     1 kg
1337    5 lbs
1338    5 lbs
Name: bag_weight_kg, Length: 1339, dtype: object

In [607]:
#Split data and convert all to kg
def convert_weight(x):
    try:
        value, unit = str(x).split(' ')
        value = float(value)
        if unit == 'kg':
            return round(value, 2)
        elif unit == 'lbs':
            return round(value * 2.20462262, 2)
        else:
            return np.nan
    except:
        return np.nan

converted_data = data.bag_weight_kg.apply(convert_weight)
converted_data

0       60.00
1       60.00
2         NaN
3       60.00
4       60.00
        ...  
1334     2.00
1335     2.00
1336     1.00
1337    11.02
1338    11.02
Name: bag_weight_kg, Length: 1339, dtype: float64

### Problem 5. Dates (1 point)
This should remind you of problem 4 but it's slightly nastier. Fix the harvest years, document the process.

While you're here, fix the expiration dates, and grading dates. Unlike the other column, these should be dates (`pd.to_datetime()` is your friend).

In [608]:
#Convert all dates only to 'YEAR' format
data.harvest_year = data.harvest_year.map(lambda x: str(x)[-4:] if pd.notna(x) else x)
data.harvest_year

0       2014
1       2014
2        NaN
3       2014
4       2014
        ... 
1334    2016
1335    2016
1336    2014
1337    2013
1338    2013
Name: harvest_year, Length: 1339, dtype: object

In [609]:
#Convert dates to d-m-Y format
data.expiration = data.expiration.map(
    lambda x: parse(x).strftime('%d-%m-%Y') if pd.notna(x) else x
)

In [610]:
data.expiration

0       03-04-2016
1       03-04-2016
2       31-05-2011
3       25-03-2016
4       03-04-2016
           ...    
1334    18-01-2017
1335    18-01-2017
1336    23-12-2015
1337    25-08-2015
1338    25-08-2015
Name: expiration, Length: 1339, dtype: object

In [611]:
#Convert dates to d-m-Y format
data.grading_date = data.grading_date.map(
    lambda x: parse(x).strftime('%d-%m-%Y') if pd.notna(x) else x
)

In [612]:
data.grading_date

0       04-04-2015
1       04-04-2015
2       31-05-2010
3       26-03-2015
4       04-04-2015
           ...    
1334    19-01-2016
1335    19-01-2016
1336    23-12-2014
1337    25-08-2014
1338    25-08-2014
Name: grading_date, Length: 1339, dtype: object

### Problem 6. Countries (1 point)
How many coffees are there with unknown countries of origin? What can you do about them?

In [613]:
data.country_of_origin.value_counts(dropna=False)

country_of_origin
Mexico                          236
Colombia                        183
Guatemala                       181
Brazil                          132
Taiwan                           75
United States (Hawaii)           73
Honduras                         53
Costa Rica                       51
Ethiopia                         44
Tanzania, United Republic Of     40
Uganda                           36
Thailand                         32
Nicaragua                        26
Kenya                            25
El Salvador                      21
Indonesia                        20
China                            16
India                            14
Malawi                           11
United States                    10
Peru                             10
Vietnam                           8
Myanmar                           8
Haiti                             6
Philippines                       5
United States (Puerto Rico)       4
Panama                            4
Laos      

In [614]:
#Read data which has not country of origin
data[data.country_of_origin.isna()]

,id,species,owner,country_of_origin,farm_name,lot_number,mill,ico_number,company,altitude,...,color,category_two_defects,expiration,certification_body,certification_address,certification_contact,unit_of_measurement,altitude_low_meters,altitude_high_meters,altitude_mean_meters
1197,1197,Arabica,racafe & cia s.c.a,NaN,NaN,NaN,NaN,3-37-1980,NaN,NaN,...,NaN,3,29-02-2012,Almacafé,e493c36c2d076bf273064f7ac23ad562af257a25,70d3c0c26f89e00fdae6fb39ff54f0d2eb1c38ab,m,NaN,NaN,NaN


In [615]:
#Print the number of records with unknown country of origin
print(f'There is {data.country_of_origin.isna().sum()} coffee/s with unknown country/ies of origin')

There is 1 coffee/s with unknown country/ies of origin


### Problem 7. Owners (1 point)
There are two suspicious columns, named `Owner`, and `Owner.1` (they're likely called something different after you solved problem 3). Do something about them. Is there any link to `Producer`?

In [616]:
#Read column 'owner'
data.owner = data.owner.str.lower()
data.owner.value_counts()

owner
juan luis alvarado romero            155
racafe & cia s.c.a                    60
exportadora de cafe condor s.a        54
kona pacific farmers cooperative      52
ipanema coffees                       50
                                    ... 
kasozi coffee farmers association      1
nitubaasa ltd                          1
mannya coffee project                  1
yidnekachew dabessa                    1
james moore                            1
Name: count, Length: 315, dtype: int64

In [617]:
#Read column 'owner_1'
data.owner_1 = data.owner_1.str.lower()
data.owner_1.value_counts()

owner_1
juan luis alvarado romero           155
racafe & cia s.c.a                   60
exportadora de cafe condor s.a       54
kona pacific farmers cooperative     52
ipanema coffees                      50
                                   ... 
yidnekachew dabessa                   1
francisco a mena                      1
specialty coffee-korea                1
nora zeas                             1
james moore                           1
Name: count, Length: 319, dtype: int64

In [618]:
#Read column 'producer'
data.producer = data.producer.str.lower()
data.producer.value_counts()

producer
la plata                             39
ipanema agrícola sa                  22
varios                               20
doi tung development project         17
ipanema agricola                     12
                                     ..
nishant gurger                        1
kigezi coffee farmers association     1
mannya coffee project                 1
idalina m. de andrade ferreira        1
coeb koperativ ekselsyo basen         1
Name: count, Length: 666, dtype: int64

### Problem 8. Coffee color by country and continent (1 point)
Create a table which shows how many coffees of each color are there in every country. Leave the missing values as they are.

**Note:** If you ask me, countries should be in rows, I prefer long tables much better than wide ones.

Now do the same for continents. You know what continent each country is located in.

In [619]:
#Group by countries of origin and color, then we count the rows in every group. Return the result in normal table with reset_index and we named the column with name='Count'
data.groupby(['country_of_origin', 'color']).size().reset_index(name='Count')

,country_of_origin,color,Count
0,Brazil,Blue-Green,14
1,Brazil,Bluish-Green,12
2,Brazil,Green,92
3,Burundi,Green,1
4,China,Green,16
...,...,...,...
74,United States (Hawaii),Green,21
75,United States (Puerto Rico),Bluish-Green,1
76,United States (Puerto Rico),Green,3
77,Vietnam,Bluish-Green,1


In [620]:
u_countries = list(set(data.country_of_origin))
u_countries

['Philippines',
 'Laos',
 'Brazil',
 'United States (Hawaii)',
 'Japan',
 'United States (Puerto Rico)',
 'Ecuador',
 'Taiwan',
 'Mexico',
 'Haiti',
 'Peru',
 'India',
 'Honduras',
 'Burundi',
 'Rwanda',
 'Kenya',
 'Vietnam',
 'Thailand',
 'Myanmar',
 'Mauritius',
 'Indonesia',
 'Cote d?Ivoire',
 'Papua New Guinea',
 'Guatemala',
 'Tanzania, United Republic Of',
 'Costa Rica',
 'United States',
 'Panama',
 'Nicaragua',
 'Colombia',
 nan,
 'China',
 'Uganda',
 'Malawi',
 'El Salvador',
 'Zambia',
 'Ethiopia']

In [621]:
continents_and_countries = []
for c in u_countries:
    try:
        country = CountryInfo(c)
        region = country.info().get("region")
        continents_and_countries.append((c, region))
    except:
        continents_and_countries.append((c, None))

continents_and_countries

[('Philippines', 'Asia'),
 ('Laos', 'Asia'),
 ('Brazil', 'Americas'),
 ('United States (Hawaii)', None),
 ('Japan', 'Asia'),
 ('United States (Puerto Rico)', None),
 ('Ecuador', 'Americas'),
 ('Taiwan', 'Asia'),
 ('Mexico', 'Americas'),
 ('Haiti', 'Americas'),
 ('Peru', 'Americas'),
 ('India', 'Asia'),
 ('Honduras', 'Americas'),
 ('Burundi', 'Africa'),
 ('Rwanda', 'Africa'),
 ('Kenya', 'Africa'),
 ('Vietnam', 'Asia'),
 ('Thailand', 'Asia'),
 ('Myanmar', None),
 ('Mauritius', 'Africa'),
 ('Indonesia', 'Asia'),
 ('Cote d?Ivoire', None),
 ('Papua New Guinea', 'Oceania'),
 ('Guatemala', 'Americas'),
 ('Tanzania, United Republic Of', None),
 ('Costa Rica', 'Americas'),
 ('United States', 'Americas'),
 ('Panama', 'Americas'),
 ('Nicaragua', 'Americas'),
 ('Colombia', 'Americas'),
 (nan, None),
 ('China', 'Asia'),
 ('Uganda', 'Africa'),
 ('Malawi', 'Africa'),
 ('El Salvador', 'Americas'),
 ('Zambia', 'Africa'),
 ('Ethiopia', 'Africa')]

### Problem 9. Ratings (1 point)
The columns `Aroma`, `Flavor`, etc., up to `Moisture` represent subjective ratings. Explore them. Show the means and range; draw histograms and / or boxplots as needed. You can even try correlations if you want. What's up with all those ratings?

### Problem 10. High-level errors (1 point)
Check the countries against region names, altitudes, and companies. Are there any discrepancies (e.g. human errors, like a region not matching the country)? Take a look at the (cleaned) altitudes; there has been a lot of preprocessing done to them. Was it done correctly?

### * Problem 11. Clean and explore at will
The dataset claimed to be clean, but we were able to discover a lot of things to fix and do better.

Play around with the data as much as you wish, and if you find variables to tidy up and clean - by all means, do that!